In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
emails = pd.read_csv('data/emails.csv')
questions = pd.read_csv('data/questions.csv')
professionals = pd.read_csv('data/professionals.csv')
comments = pd.read_csv('data/comments.csv')
tag_users = pd.read_csv('data/tag_users.csv')
group_memberships = pd.read_csv('data/group_memberships.csv')
tags = pd.read_csv('data/tags.csv')
students = pd.read_csv('data/students.csv')
groups = pd.read_csv('data/groups.csv')
tag_questions = pd.read_csv('data/tag_questions.csv')
matches = pd.read_csv('data/matches.csv')
answers = pd.read_csv('data/answers.csv')
school_memberships = pd.read_csv('data/school_memberships.csv')

In [3]:
emails.head()

,emails_id,emails_recipient_id,emails_date_sent,emails_frequency_level
0,2337714,0c673e046d824ec0ad0ebe012a0673e4,2018-12-07 01:05:40 UTC+0000,email_notification_daily
1,2336077,0c673e046d824ec0ad0ebe012a0673e4,2018-12-06 01:14:15 UTC+0000,email_notification_daily
2,2314660,0c673e046d824ec0ad0ebe012a0673e4,2018-11-17 00:38:27 UTC+0000,email_notification_daily
3,2312639,0c673e046d824ec0ad0ebe012a0673e4,2018-11-16 00:32:19 UTC+0000,email_notification_daily
4,2299700,0c673e046d824ec0ad0ebe012a0673e4,2018-11-08 00:16:40 UTC+0000,email_notification_daily


In [4]:
comments.head()

,comments_id,comments_author_id,comments_parent_content_id,comments_date_added,comments_body
0,f30250d3c2ca489db1afa9b95d481e08,9fc88a7c3323466dbb35798264c7d497,b476f9c6d9cd4c50a7bacdd90edd015a,2019-01-31 23:39:40 UTC+0000,"First, you speak to recruiters. They are train..."
1,ca9bfc4ba9464ea383a8b080301ad72c,de2415064b9b445c8717425ed70fd99a,ef4b6ae24d1f4c3b977731e8189c7fd7,2019-01-31 20:30:47 UTC+0000,Most large universities offer study abroad pro...
2,c354f6e33956499aa8b03798a60e9386,6ed20605002a42b0b8e3d6ac97c50c7f,ca7a9d7a95df471c816db82ee758f57d,2019-01-31 18:44:04 UTC+0000,"First, I want to put you at ease that the oppo..."
3,73a6223948714c5da6231937157e4cb7,d02f6d9faac24997a7003a59e5f34bd3,c7a88aa76f5f49b0830bfeb46ba17e4d,2019-01-31 17:53:28 UTC+0000,Your question submission was great! I just wan...
4,55a89a9061d44dd19569c45f90a22779,e78f75c543e84e1c94da1801d8560f65,c7a88aa76f5f49b0830bfeb46ba17e4d,2019-01-31 14:51:53 UTC+0000,Thank you. I'm new to this site. I'm sorry if ...


In [6]:
questions.head()

,questions_id,questions_author_id,questions_date_added,questions_title,questions_body
0,332a511f1569444485cf7a7a556a5e54,8f6f374ffd834d258ab69d376dd998f5,2016-04-26 11:14:26 UTC+0000,Teacher career question,What is a maths teacher? what is a ma...
1,eb80205482e4424cad8f16bc25aa2d9c,acccbda28edd4362ab03fb8b6fd2d67b,2016-05-20 16:48:25 UTC+0000,I want to become an army officer. What can I d...,I am Priyanka from Bangalore . Now am in 10th ...
2,4ec31632938a40b98909416bdd0decff,f2c179a563024ccc927399ce529094b5,2017-02-08 19:13:38 UTC+0000,Will going abroad for your first job increase ...,I'm planning on going abroad for my first job....
3,2f6a9a99d9b24e5baa50d40d0ba50a75,2c30ffba444e40eabb4583b55233a5a4,2017-09-01 14:05:32 UTC+0000,To become a specialist in business management...,i hear business management is a hard way to ge...
4,5af8880460c141dbb02971a1a8369529,aa9eb1a2ab184ebbb00dc01ab663428a,2017-09-01 02:36:54 UTC+0000,Are there any scholarships out there for stude...,I'm trying to find scholarships for first year...


In [7]:
matches.head()

,matches_email_id,matches_question_id
0,1721939,332a511f1569444485cf7a7a556a5e54
1,1665388,332a511f1569444485cf7a7a556a5e54
2,1636634,332a511f1569444485cf7a7a556a5e54
3,1635498,332a511f1569444485cf7a7a556a5e54
4,1620298,332a511f1569444485cf7a7a556a5e54


In [8]:
tag_questions.head()

,tag_questions_tag_id,tag_questions_question_id
0,28930,cb43ebee01364c68ac61d347a393ae39
1,28930,47f55e85ce944242a5a347ab85a8ffb4
2,28930,ccc30a033a0f4dfdb2eb987012f25792
3,28930,e30b274e48d741f7bf50eb5e7171a3c0
4,28930,3d22742052df4989b311b4195cbb0f1a


In [9]:
tags.head()

,tags_tag_id,tags_tag_name
0,27490,college
1,461,computer-science
2,593,computer-software
3,27292,business
4,18217,doctor


In [15]:
tag_questions.tag_questions_tag_id.value_counts().head()

27490    3744
129      1566
89       1324
54       1083
27292     989
Name: tag_questions_tag_id, dtype: int64

In [33]:
(tag_questions.groupby('tag_questions_question_id').count().reset_index()
 .groupby('tag_questions_tag_id').count().reset_index().rename(
     columns={'tag_questions_question_id': 'number_of_questions',
              'tag_questions_tag_id': 'number_of_tags'
             })).head(10)

,number_of_tags,number_of_questions
0,1,5056
1,2,4652
2,3,4510
3,4,3524
4,5,2466
5,6,1480
6,7,713
7,8,574
8,9,133
9,10,63


In [41]:
tag_questions.set_index(['tag_questions_question_id', 'tag_questions_tag_id']).head()

,
tag_questions_question_id,tag_questions_tag_id
cb43ebee01364c68ac61d347a393ae39,28930
47f55e85ce944242a5a347ab85a8ffb4,28930
ccc30a033a0f4dfdb2eb987012f25792,28930
e30b274e48d741f7bf50eb5e7171a3c0,28930
3d22742052df4989b311b4195cbb0f1a,28930


In [69]:
tag_groups_dict = tag_questions.groupby('tag_questions_question_id').groups

In [80]:
i = 0
for question, tags in tag_groups_dict.items():
    tag_groups_dict[question] = list(tags)
    
    if (i < 5):
        print(question, ':', list(tags))
    i += 1

0003e7bf48f24b5c985f8fce96e611f3 : [12839, 23735, 29372, 48543, 59166, 59956]
0006609dd4da40dcaa5a83e0499aba14 : [42995, 43792]
000af224bc2f4e94a19f8b62ba279cc4 : [7919, 32359]
000b30fb534b41f7b716fa9ebf9c3f35 : [5633, 30173, 34018, 74570]
0018752e44b44e26bb74a0a43232b4d6 : [21722, 57042]


In [82]:
emails.head()

,emails_id,emails_recipient_id,emails_date_sent,emails_frequency_level
0,2337714,0c673e046d824ec0ad0ebe012a0673e4,2018-12-07 01:05:40 UTC+0000,email_notification_daily
1,2336077,0c673e046d824ec0ad0ebe012a0673e4,2018-12-06 01:14:15 UTC+0000,email_notification_daily
2,2314660,0c673e046d824ec0ad0ebe012a0673e4,2018-11-17 00:38:27 UTC+0000,email_notification_daily
3,2312639,0c673e046d824ec0ad0ebe012a0673e4,2018-11-16 00:32:19 UTC+0000,email_notification_daily
4,2299700,0c673e046d824ec0ad0ebe012a0673e4,2018-11-08 00:16:40 UTC+0000,email_notification_daily


In [110]:
professionals.professionals_industry.isna().mean()

0.0915032679738562

In [126]:
qa = questions.merge(answers, how='right', left_on='questions_id', right_on='answers_question_id')

In [125]:
qap = qa.merge(professionals, left_on='answers_author_id', right_on='professionals_id')
qap = qap[['questions_id', 'questions_title', 'questions_date_added', 'answers_date_added',
           'professionals_id', 'professionals_industry']]
qap.head()

,questions_id,questions_title,questions_date_added,answers_date_added,professionals_id,professionals_industry
0,332a511f1569444485cf7a7a556a5e54,Teacher career question,2016-04-26 11:14:26 UTC+0000,2016-04-29 19:40:14 UTC+0000,36ff3b3666df400f956f8335cf53e09e,Mental Health Care
1,0f1d6a4f276c4a05878dd48e03e52289,what kind of college could i go to for a soc...,2016-05-19 22:16:25 UTC+0000,2016-07-31 15:35:54 UTC+0000,36ff3b3666df400f956f8335cf53e09e,Mental Health Care
2,0149c6d63e214040b44d4a3789bb00ba,What is the best way to prepare for studying e...,2018-04-12 17:13:45 UTC+0000,2018-04-15 23:08:46 UTC+0000,36ff3b3666df400f956f8335cf53e09e,Mental Health Care
3,acc611cfb5c44daa8a3d7d65dfffa5ff,How should I prepare myself for my job search ...,2018-08-14 04:49:33 UTC+0000,2018-08-14 10:37:01 UTC+0000,36ff3b3666df400f956f8335cf53e09e,Mental Health Care
4,b44409c0c83a43578f4f2d0a0e4d8ead,What majors do you recommend in order to be an...,2016-12-07 20:43:41 UTC+0000,2016-12-08 02:26:05 UTC+0000,36ff3b3666df400f956f8335cf53e09e,Mental Health Care


In [128]:
emails.head()

,emails_id,emails_recipient_id,emails_date_sent,emails_frequency_level
0,2337714,0c673e046d824ec0ad0ebe012a0673e4,2018-12-07 01:05:40 UTC+0000,email_notification_daily
1,2336077,0c673e046d824ec0ad0ebe012a0673e4,2018-12-06 01:14:15 UTC+0000,email_notification_daily
2,2314660,0c673e046d824ec0ad0ebe012a0673e4,2018-11-17 00:38:27 UTC+0000,email_notification_daily
3,2312639,0c673e046d824ec0ad0ebe012a0673e4,2018-11-16 00:32:19 UTC+0000,email_notification_daily
4,2299700,0c673e046d824ec0ad0ebe012a0673e4,2018-11-08 00:16:40 UTC+0000,email_notification_daily


In [145]:
matches_groups_dict = matches.groupby('matches_email_id').groups

i = 0
for email, questions in matches_groups_dict.items():
    matches_groups_dict[email] = list(questions)
    
    if (i < 5):
        print(email, ':', list(questions))
    i += 1

106 : [3348214]
107 : [3348215]
108 : [3348216]
110 : [3348217]
277 : [1408486]


In [147]:
email_questions_dict = matches_groups_dict

In [146]:
counts.matches_question_id.value_counts().head()

3     986681
1     645047
2     126182
19     12320
4       5535
Name: matches_question_id, dtype: int64

In [129]:
matches.head()

,matches_email_id,matches_question_id
0,1721939,332a511f1569444485cf7a7a556a5e54
1,1665388,332a511f1569444485cf7a7a556a5e54
2,1636634,332a511f1569444485cf7a7a556a5e54
3,1635498,332a511f1569444485cf7a7a556a5e54
4,1620298,332a511f1569444485cf7a7a556a5e54
